<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 4 Sprint 3 Assignment 1*

# Recurrent Neural Networks and Long Short Term Memory (LSTM)

![Monkey at a typewriter](https://upload.wikimedia.org/wikipedia/commons/thumb/3/3c/Chimpanzee_seated_at_typewriter.jpg/603px-Chimpanzee_seated_at_typewriter.jpg)

It is said that [infinite monkeys typing for an infinite amount of time](https://en.wikipedia.org/wiki/Infinite_monkey_theorem) will eventually type, among other things, the complete works of Wiliam Shakespeare. Let's see if we can get there a bit faster, with the power of Recurrent Neural Networks and LSTM.

This text file contains the complete works of Shakespeare: https://www.gutenberg.org/files/100/100-0.txt

Use it as training data for an RNN - you can keep it simple and train character level, and that is suggested as an initial approach.

Then, use that trained RNN to generate Shakespearean-ish text. Your goal - a function that can take, as an argument, the size of text (e.g. number of characters or lines) to generate, and returns generated text of that size.

Note - Shakespeare wrote an awful lot. It's OK, especially initially, to sample/use smaller data and parameters, so you can have a tighter feedback loop when you're trying to get things running. Then, once you've got a proof of concept - start pushing it more!

In [1]:
import requests
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding
from tensorflow.keras.layers import LSTM
from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.optimizers import RMSprop

In [2]:
import numpy as np
import random
import sys
import os

In [3]:
data = []
with open("shakespeare.txt", 'r', encoding='utf-8') as f:
    data.append(f.read())

In [4]:
data[0][:500]

'\ufeff\nProject Gutenberg’s The Complete Works of William Shakespeare, by William\nShakespeare\n\nThis eBook is for the use of anyone anywhere in the United States and\nmost other parts of the world at no cost and with almost no restrictions\nwhatsoever.  You may copy it, give it away or re-use it under the terms\nof the Project Gutenberg License included with this eBook or online at\nwww.gutenberg.org.  If you are not located in the United States, you’ll\nhave to check the laws of the country where you are l'

In [5]:
text = " ".join(data)

# Unique Characters
chars = list(set(text))

# Lookup Tables
char_int = {c:i for i, c in enumerate(chars)} 
int_char = {i:c for i, c in enumerate(chars)} 

In [6]:
maxlen = 40
step = 5

encoded = [char_int[c] for c in text]

sequences = [] # Each element is 40 chars long
next_char = [] # One element for each sequence

for i in range(0, len(encoded) - maxlen, step):
    
    sequences.append(encoded[i : i + maxlen])
    next_char.append(encoded[i + maxlen])
    
print('sequences: ', len(sequences))

x = np.zeros((len(sequences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sequences),len(chars)), dtype=np.bool)

for i, sequence in enumerate(sequences):
    for t, char in enumerate(sequence):
        x[i,t,char] = 1
        
    y[i, next_char[i]] = 1

sequences:  1114623


In [7]:
sequences = sequences[:100000]

In [8]:
x = np.zeros((len(sequences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sequences),len(chars)), dtype=np.bool)

for i, sequence in enumerate(sequences):
    for t, char in enumerate(sequence):
        x[i,t,char] = 1
        
    y[i, next_char[i]] = 1

In [9]:
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars)), dropout=.2))
model.add(Dense(len(chars), activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')

In [10]:
def sample(preds):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / 1
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [11]:
def on_epoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    
    print()
    print('----- Generating text after Epoch: %d' % epoch)
    
    start_index = random.randint(0, len(text) - maxlen - 1)
    
    generated = ''
    
    sentence = text[start_index: start_index + maxlen]
    generated += sentence
    
    print('----- Generating with seed: "' + sentence + '"')
    sys.stdout.write(generated)
    
    for i in range(400):
        x_pred = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_int[char]] = 1
            
        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds)
        next_char = int_char[next_index]
        
        sentence = sentence[1:] + next_char
        
        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()


print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

In [25]:
import tensorflow as tf


tf.compat.v1.ConfigProto.gpu_options.allow_growth = True

AttributeError: 'google.protobuf.pyext._message.FieldProperty' object has no attribute 'allow_growth'

In [23]:
tf.config

AttributeError: 'NoneType' object has no attribute 'config'

In [12]:
model.fit(x, y,
          batch_size=2000,
          epochs=15,
          callbacks=[print_callback])

Train on 100000 samples
Epoch 1/15
 98000/100000 [============================>.] - ETA: 0s - loss: 3.5632
----- Generating text after Epoch: 0
----- Generating with seed: "That under covert and convenient seeming"
That under covert and convenient seemingeiTesaemhen
 eeecnLmudrt
gitShlTlneQdHlb ,r.t N r etr d
eureai .h[sato’feR.oSevmb
eLu
c, enho,ve wLStt1geiesbgplnwun ha.dlrdldscr?o6a
 etro?tmhf  vfuosHee,rsk ren wetI|nlOl  NTeh tsC   oavl ybt Oh i “elSn  ,eh ,  ,hRAue TTri rLfT eu ieesut cw,tak .atr re heai 
 eemsldmcfe
“ahhpnK o
he, i n ttry,t nrp,lel c rt. icRhse; uaye
hNMawy IOei dai;a  os h,p,hgegoNvtIees ea s}m?o  snoI nLelherYutesdiftt  Re
100000/100000 [==============================] - 19s 191us/sample - loss: 3.5563
Epoch 2/15
 98000/100000 [============================>.] - ETA: 0s - loss: 3.1942
----- Generating text after Epoch: 1
----- Generating with seed: "an, shrill and sound,
And all is semblat"
an, shrill and sound,
And all is semblatt 
 ot. me_t re  me, r  ah g snE

In [13]:
maxlen = 40
step = 5

encoded = [char_int[c] for c in text]

sequences = [] # Each element is 40 chars long
next_char = [] # One element for each sequence

for i in range(0, len(encoded) - maxlen, step):
    
    sequences.append(encoded[i : i + maxlen])
    next_char.append(encoded[i + maxlen])
    
print('sequences: ', len(sequences))

x = np.zeros((len(sequences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sequences),len(chars)), dtype=np.bool)

for i, sequence in enumerate(sequences):
    for t, char in enumerate(sequence):
        x[i,t,char] = 1
        
    y[i, next_char[i]] = 1

sequences:  1114623


In [14]:
model.fit(x, y,
          batch_size=2000,
          epochs=15,
          callbacks=[print_callback])

Train on 1114623 samples
Epoch 1/15
1114000/1114623 [============================>.] - ETA: 0s - loss: 2.4279
----- Generating text after Epoch: 0
----- Generating with seed: " our seeing, goe we hence,
Right joyfull"
 our seeing, goe we hence,
Right joyfull a pratt, yas nge; ghell
     Thes soor my uame in wole hent cooi wid.

THKOGA.
PSor arg'grelt
    ir is  mout a’d thev din cringom oulitis; ary;  n arith m srarsels? Sou’s mo, ank hateatis qury;

 _UHICEL._]athy, aWsme wov'e sblbt bar; ar Myse nitherd INisks'nd,

    houzheclsenth woucurt, orv, thange tit
yam norm whall ofh, wher, peary;
hy oul as whing aulesi he sesto  numgh,
 T  nin to sa fase 
1114623/1114623 [==============================] - 51s 46us/sample - loss: 2.4278
Epoch 2/15
1114000/1114623 [============================>.] - ETA: 0s - loss: 2.3261
----- Generating text after Epoch: 1
----- Generating with seed: "on the instant that she was accus’d,
   "
on the instant that she was accus’d,
    shit mbelir ly. Lelly ham

# Resources and Stretch Goals

## Stretch goals:
- Refine the training and generation of text to be able to ask for different genres/styles of Shakespearean text (e.g. plays versus sonnets)
- Train a classification model that takes text and returns which work of Shakespeare it is most likely to be from
- Make it more performant! Many possible routes here - lean on Keras, optimize the code, and/or use more resources (AWS, etc.)
- Revisit the news example from class, and improve it - use categories or tags to refine the model/generation, or train a news classifier
- Run on bigger, better data

## Resources:
- [The Unreasonable Effectiveness of Recurrent Neural Networks](https://karpathy.github.io/2015/05/21/rnn-effectiveness/) - a seminal writeup demonstrating a simple but effective character-level NLP RNN
- [Simple NumPy implementation of RNN](https://github.com/JY-Yoon/RNN-Implementation-using-NumPy/blob/master/RNN%20Implementation%20using%20NumPy.ipynb) - Python 3 version of the code from "Unreasonable Effectiveness"
- [TensorFlow RNN Tutorial](https://github.com/tensorflow/models/tree/master/tutorials/rnn) - code for training a RNN on the Penn Tree Bank language dataset
- [4 part tutorial on RNN](http://www.wildml.com/2015/09/recurrent-neural-networks-tutorial-part-1-introduction-to-rnns/) - relates RNN to the vanishing gradient problem, and provides example implementation
- [RNN training tips and tricks](https://github.com/karpathy/char-rnn#tips-and-tricks) - some rules of thumb for parameterizing and training your RNN